In [1]:
import json

from torch.ao.nn.quantized.functional import threshold

# open and read content job_skills
with open("job_skill.json", "r", encoding='utf-8') as f:
    job_skills = json.load(f)

job_skills = [job for job in job_skills.values()]
import pandas as pd
import json

# open and read content job_skills
with open("job_skill.json", "r", encoding='utf-8') as f:
    job_skills = json.load(f)

all_job_skills = [skill["skill"] for profession in job_skills.values() for skill in profession]
unique_job_skills = list(set(all_job_skills))

print(unique_job_skills)
print(len(all_job_skills), len(unique_job_skills))

['1с управление производственным предприятием', 'wireframing', 'go', 'java core', 'bash', 'oracle pl sql', 'viper', 'cisco', 'api documentation', 'data governance', '.net framework', 'core data', 'matlab', 'настройка по', 'etl', 'ui design', 'data pipelines', 'scss', 'webpack', 'chef', 'backup and recovery', 'dbt', 'pyspark', 'azure functions', 'lambda', 'dynamodb', '1с комплексная автоматизация', 'bigquery', 'unity', 'functional testing', 'xcode', 'java ee', 'java se', 'information architecture', 'quantitative research', 'html', 'cocoapods', 'unreal engine', 'angular', 'responsive design', 'yii2', 'xml', 'erp системы на базе 1с', 'powershell', 'user management', 'prototyping', 'c#', 'usability testing', 'test case', 'ssis', 'tcp ip', 'vue.js', 'sql', 'combine', 'python', 'субд', 'jetpack compose', 'azure devops', 'numpy', 'firebase', 'product management', 'cybersecurity', 'siem tools', 'android', 'helm', 'git', 'node.js', 'accessibility', 'elasticsearch', 'razor', 'mysql', 'azure', '1

In [2]:
import pandas as pd
import ast

df = pd.read_csv("courses_final.csv")

df['skills'] = df['skills'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

all_courses_skills = df['skills'].explode()
unique_course_skills = all_courses_skills.dropna().unique().tolist()

print(unique_course_skills)
print(len(all_courses_skills), len(unique_course_skills))


['NumPy', 'Data Manipulation', 'File Management', 'Data Structures', 'Application Programming Interface (API)', 'Data Analysis', 'Programming Principles', 'Web Scraping', 'Restful API', 'Object Oriented Programming (OOP)', 'Computer Programming', 'Jupyter', 'Python Programming', 'Data Import/Export', 'Pandas (Python Package)', 'Development Environment', 'Computational Thinking', 'Integrated Development Environments', 'Debugging', 'Algorithms', 'Problem Management', 'Network Protocols', 'Database Design', 'SQL', 'Data Collection', 'Data Visualization Software', 'Data Processing', 'Data Modeling', 'Test Driven Development (TDD)', 'Software Testing', 'Unit Testing', 'Plotly', 'Docker (Software)', 'Scripting', 'Git (Version Control System)', 'Data Ethics', 'Agile Methodology', 'Flask (Web Framework)', 'Web Development', 'Matplotlib', 'Statistical Modeling', 'Supervised Learning', 'Predictive Modeling', 'Regression Analysis', 'Data Cleansing', 'Data Pipelines', 'Descriptive Statistics', 'Fe

In [19]:
import re

def clean_skill(skill: str) -> str:
    cleaned = re.sub(r'\s*\(.*?\)', '', skill)
    return cleaned.strip().lower()

cleaned_course_skills = list(set([clean_skill(skill) for skill in unique_course_skills]))
print(cleaned_course_skills)

['electronic hardware', 'data analysis', 'text mining', 'computer-aided design', 'heat maps', 'matlab', 'business valuation', 'hadoop distributed file system', 'interface design', 'feed forward', 'artificial neural networks', 'radiology', 'sales', 'integrated development environments', 'semantic web', 'research design', 'cyber security strategy', 'opengl shading language', 'market trend', 'statistical software', 'handwriting recognition', 'cloud foundry', 'computational logic', 'business intelligence tools', 'clinical trials', 'functional testing', 'soil science', 'musical composition', 'mobile development', 'quantitative research', 'recommender systems', 'distributed computing', 'unified modeling language', 'lean manufacturing', 'data transmissions', 'game design', 'data science', 'information management', 'technical communication', 'ggplot2', 'business analysis', 'cloud infrastructure', 'information technology', 'mathematical finance', 'c#', 'sqlite', 'directed acyclic graph', 'linux

In [17]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2')
# model = SentenceTransformer('multilingual-e5-large')

In [20]:
course_embeddings = model.encode(cleaned_course_skills, normalize_embeddings=True)
job_embeddings = model.encode(unique_job_skills, normalize_embeddings=True)


In [22]:
from sklearn.metrics.pairwise import cosine_similarity

similarity_matrix = cosine_similarity(job_embeddings, course_embeddings)

In [24]:
import numpy as np

skill_map = {}
threshold = 0.85
for i, prof_skill in enumerate(unique_job_skills):
    top_indices = [
        idx for idx in np.argsort(similarity_matrix[i])[::-1]
            if threshold < similarity_matrix[i][idx] and not np.isclose(similarity_matrix[i][idx], 1.0)
    ][:3]
    if top_indices:
        skill_map[prof_skill] = [
            (cleaned_course_skills[idx], round(float(similarity_matrix[i][idx]), 2))
            for idx in top_indices
        ]

print(skill_map)
for prof_skill, matches in skill_map.items():
    print(f"Skill from job: {prof_skill}")
    for course_skill, score in matches:
        print(f"  --> {course_skill} (accuracy: {score})")
    print()

{'go': [('git', 0.95)], 'data pipelines': [('data pipeline', 0.99)], '1с комплексная автоматизация': [('automation', 0.91)], 'java ee': [('jest', 0.94)], 'java se': [('jest', 0.88)], 'html': [('html5', 0.89)], 'responsive design': [('responsive web design', 0.89)], 'prototyping': [('rapid prototyping', 0.9)], 'tcp ip': [('tcp/ip', 0.96)], 'python': [('python programming', 0.91)], 'cybersecurity': [('cyber security', 1.0), ('cyber security strategy', 0.94), ('cyber security assessment', 0.91)], 'siem tools': [('tooling', 0.86)], 'android': [('android applications', 0.89)], 'git': [('go', 0.95)], 'linux': [('linux kernel', 0.86)], 'data cleaning': [('data cleansing', 0.94)], 'windows': [('microsoft windows', 0.89)], 'financial modeling': [('financial analysis', 0.87)], 'ms sql': [('sql', 0.87)], 'presentation': [('presentations', 0.97)], '3d modeling basics': [('3d modeling', 0.95)], 'cloud architecture': [('cloud computing architecture', 0.97), ('cloud computing', 0.92), ('cloud enginee

In [31]:
from collections import defaultdict

reverse_skill_map = defaultdict(list)

for prof_skill, course_matches in skill_map.items():
    for course_skill, score in course_matches:
        reverse_skill_map[course_skill].append(prof_skill)

for course in reverse_skill_map:
    reverse_skill_map[course].sort(key=lambda x: x[1], reverse=True)

reverse_skill_map = dict(reverse_skill_map)

print(reverse_skill_map)

{'git': ['go'], 'data pipeline': ['data pipelines'], 'automation': ['1с комплексная автоматизация'], 'jest': ['java ee', 'java se', 'java'], 'html5': ['html'], 'responsive web design': ['responsive design'], 'rapid prototyping': ['prototyping'], 'tcp/ip': ['tcp ip'], 'python programming': ['python'], 'cyber security': ['cybersecurity'], 'cyber security strategy': ['cybersecurity'], 'cyber security assessment': ['cybersecurity'], 'tooling': ['siem tools'], 'android applications': ['android'], 'go': ['git'], 'linux kernel': ['linux'], 'data cleansing': ['data cleaning'], 'microsoft windows': ['windows'], 'financial analysis': ['financial modeling'], 'sql': ['ms sql'], 'presentations': ['presentation'], '3d modeling': ['3d modeling basics'], 'cloud computing architecture': ['cloud architecture'], 'cloud computing': ['cloud architecture'], 'cloud engineering': ['cloud architecture'], 'security systems': ['security'], 'security policies': ['security'], 'computer security': ['security'], 'ci

In [11]:
with open('skill_mapping.json', 'w', encoding='utf-8') as f:
    json.dump(skill_map, f, ensure_ascii=False, indent=2)